<a href="https://colab.research.google.com/github/shubhamlatkar/Stock-Predicition/blob/master/TweetAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gettting the tweet data

In [0]:
import time
import datetime
from urllib.request import urlopen
import requests
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd

In [0]:
start_date = datetime.date(2019, 1, 1)
end_date = datetime.date(2019, 6, 1)
date_list = [ start_date + datetime.timedelta(n) for n in range(int ((end_date - start_date).days))]
date_list.reverse()
date_list = [x.strftime("%Y%m%d") for x in date_list]
print(date_list)

['20190531', '20190530', '20190529', '20190528', '20190527', '20190526', '20190525', '20190524', '20190523', '20190522', '20190521', '20190520', '20190519', '20190518', '20190517', '20190516', '20190515', '20190514', '20190513', '20190512', '20190511', '20190510', '20190509', '20190508', '20190507', '20190506', '20190505', '20190504', '20190503', '20190502', '20190501', '20190430', '20190429', '20190428', '20190427', '20190426', '20190425', '20190424', '20190423', '20190422', '20190421', '20190420', '20190419', '20190418', '20190417', '20190416', '20190415', '20190414', '20190413', '20190412', '20190411', '20190410', '20190409', '20190408', '20190407', '20190406', '20190405', '20190404', '20190403', '20190402', '20190401', '20190331', '20190330', '20190329', '20190328', '20190327', '20190326', '20190325', '20190324', '20190323', '20190322', '20190321', '20190320', '20190319', '20190318', '20190317', '20190316', '20190315', '20190314', '20190313', '20190312', '20190311', '20190310', '20

In [0]:
suffix = {'National Stock Exchange': '.NS'}
url = "http://www.reuters.com/finance/stocks/company-news/" + "INFY" + ".NS"
print(url)
has_content = 0
repeat_times = 4   

http://www.reuters.com/finance/stocks/company-news/INFY.NS


In [0]:
df = pd.DataFrame(columns = ['ticker' , 'timestamp' , 'title' , 'newsType'])
df.describe()

,ticker,timestamp,title,newsType
count,0,0,0,0
unique,0,0,0,0


In [0]:
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
has_content = len(soup.find_all("div", {'class': ['topStory', 'feature']}))
print(has_content) 

10


In [0]:

ticker = "INFY"
if has_content > 0:
  missing_days = 0
  for timestamp in date_list:
    new_time = timestamp[4:] + timestamp[:4]
    page = requests.get(url + "?date=" + new_time)
    soup = BeautifulSoup(page.content, 'html.parser')
    content = soup.find_all("div", {'class': ['topStory', 'feature']})
#   fout = open('./news_reuters.csv', 'a+')
    j = 0  
    for i in range(len(content)):
      title = content[i].h2.get_text().replace(",", " ").replace("\n", " ")
      body = content[i].p.get_text().replace(",", " ").replace("\n", " ")
      
      if i == 0 and soup.find_all("div", class_="topStory"):
        news_type = 'topStory'
      else:
        news_type = 'normal'

      df.loc[j] = [ticker, timestamp, title, news_type] 
      j += 1
      #fout.write(str(','.join([timestamp, title, body, news_type]).encode('utf-8') ))
      #fout.write('\n')
    #fout.close()
    print(url + "?date=" + new_time+"     "+str(1))
    

In [0]:
print(df.head())

  ticker timestamp                                              title  newsType
0   INFY  20190108  India forms panel to deepen digital payments  ...  topStory
1   INFY  20190108  India forms panel to deepen digital payments  ...    normal
2   INFY  20190111  India's Infosys Q3 profit falls 29.6 pct  miss...    normal
3   INFY  20190111  Indian shares end lower  all eyes on Infosys r...    normal
4   INFY  20190111  Indian shares subdued ahead of Infosys results...    normal


MoneyControl News Scapping

In [0]:
from bs4 import BeautifulSoup
import requests
import tweepy 
import json
import csv
import sys

In [0]:
def get_all_news():
  df = pd.DataFrame(columns = ['Timestamp' , 'ImgSrc' , 'Title' , 'Text'])
  i=0
  for loop in  range(0,100):
    url  = "https://www.moneycontrol.com/news/business/stocks/page-" + str(loop) + "/"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    for item in soup.find_all('li'  , class_ = 'clearfix'):
        #print(item.find('img').get('src'))
        #print(item.find("span").string)
        #print(item.find("h2").find('a').get('title'))
        #print(item.find("p").string)

        df.loc[i] = [ 
                      str(item.find("span").string) ,
                      str(item.find('img').get('src')) ,
                      str(item.find("h2").find('a').get('title')) , 
                      str(item.find("p").string)
                    ]
        i+=1
  df = df.set_index('Timestamp') 
  return df

In [0]:
df = get_all_news()
print(df.tail())

                                                                        ImgSrc  ...                                               Text
Timestamp                                                                       ...                                                   
March 07, 2019 12:28 PM IST  https://static-news.moneycontrol.com/static-mc...  ...  Daawat Foods serves as manufacturing arm of LT...
March 07, 2019 12:17 PM IST  https://static-news.moneycontrol.com/static-mc...  ...  Post general election and stability in domesti...
March 07, 2019 11:54 AM IST  https://static-news.moneycontrol.com/static-mc...  ...  CLSA also reiterated its Buy rating on Zee Ent...
March 07, 2019 11:00 AM IST  https://static-news.moneycontrol.com/static-mc...  ...  The orders are in the range of Rs 2,500 to Rs ...
March 07, 2019 09:47 AM IST  https://static-news.moneycontrol.com/static-mc...  ...  Promoter ARIIL Trust raises stake in company b...

[5 rows x 3 columns]


In [0]:
# df.to_csv('News.csv' , index = True)

NameError: ignored

In [0]:
# df.describe()

NameError: ignored

In [0]:
# print(df.head())

NameError: ignored

In [0]:
# print(df.tail())

                                                                        ImgSrc  ...                                               Text
Timestamp                                                                       ...                                                   
March 07, 2019 12:28 PM IST  https://static-news.moneycontrol.com/static-mc...  ...  Daawat Foods serves as manufacturing arm of LT...
March 07, 2019 12:17 PM IST  https://static-news.moneycontrol.com/static-mc...  ...  Post general election and stability in domesti...
March 07, 2019 11:54 AM IST  https://static-news.moneycontrol.com/static-mc...  ...  CLSA also reiterated its Buy rating on Zee Ent...
March 07, 2019 11:00 AM IST  https://static-news.moneycontrol.com/static-mc...  ...  The orders are in the range of Rs 2,500 to Rs ...
March 07, 2019 09:47 AM IST  https://static-news.moneycontrol.com/static-mc...  ...  Promoter ARIIL Trust raises stake in company b...

[5 rows x 3 columns]


In [0]:
# for i in range(0 , (len(df) - 2420)):
#   m , d , c , y , t , k  = df.Timestamp[i].split(" ")
#   print(m)
#   print(d)
#   print(c)
#   print(y)
#   print(t)
#   print(k)
#   d = d[0] + "" +d[1]
#   print(type(m))
#   date = m+" "+str(d)+" "+str(c)+" "+str(y)+" "+str(t)+""
#   print(date)
#   df.Timestamp[i] =  datetime.datetime.strptime( date , '%B %d %Y %I:%M %p')

In [0]:
df = pd.read_csv("News.csv")
myFormat = "%Y-%m-%d"
for i in range(len(df)):
  m , d , c , y , t , k  = str(df.Timestamp[i]).split(" ")
  d = d[0] + "" +d[1]
  date = m+" "+str(d)+" "+str(c)+" "+str(y)+" "+str(t)+""
  df.Timestamp[i] =  datetime.datetime.strptime( date , '%B %d %Y %I:%M %p')
  df.Timestamp[i] = df.Timestamp[i].strftime(myFormat)
df.set_index("Timestamp" , inplace = True)
print(df.head())

                                                       ImgSrc  ...                                               Text
Timestamp                                                      ...                                                   
2019-06-15  https://static-news.moneycontrol.com/static-mc...  ...  Investors may tender their shares to enjoy ret...
2019-06-14  https://static-news.moneycontrol.com/static-mc...  ...  All eyes remain on the crude movement as well ...
2019-06-14  https://static-news.moneycontrol.com/static-mc...  ...  The European agency inspected Bengaluru unit i...
2019-06-14  https://static-news.moneycontrol.com/static-mc...  ...  Prabhudas Lilladher recommended accumulate rat...
2019-06-14  https://static-news.moneycontrol.com/static-mc...  ...  Prabhudas Lilladher recommended accumulate rat...

[5 rows x 3 columns]


                                                                ImgSrc  ...                                               Text
Timestamp                                                               ...                                                   
2019-06-15 10:49:00  https://static-news.moneycontrol.com/static-mc...  ...  Investors may tender their shares to enjoy ret...
2019-06-14 23:44:00  https://static-news.moneycontrol.com/static-mc...  ...  All eyes remain on the crude movement as well ...
2019-06-14 16:45:00  https://static-news.moneycontrol.com/static-mc...  ...  The European agency inspected Bengaluru unit i...
2019-06-14 15:53:00  https://static-news.moneycontrol.com/static-mc...  ...  Prabhudas Lilladher recommended accumulate rat...
2019-06-14 15:47:00  https://static-news.moneycontrol.com/static-mc...  ...  Prabhudas Lilladher recommended accumulate rat...

[5 rows x 3 columns]


In [0]:
# df = df.reset_index()
# print(str(df.Timestamp[0]))
# df.Timestamp[0] = datetime.datetime.strptime( str(df.Timestamp[0]) , '%Y-%m-%d').date()
# myFormat = "%Y-%m-%d"
# print ( df.Timestamp[0].strftime(myFormat))

AttributeError: ignored

In [0]:
# for i in range(len(df)):
#   df.Timestamp[i] = df.Timestamp[i].strftime(myFormat)
# print(df.head())  

    Timestamp  ...                                               Text
0  2019-06-15  ...  Investors may tender their shares to enjoy ret...
1  2019-06-14  ...  All eyes remain on the crude movement as well ...
2  2019-06-14  ...  The European agency inspected Bengaluru unit i...
3  2019-06-14  ...  Prabhudas Lilladher recommended accumulate rat...
4  2019-06-14  ...  Prabhudas Lilladher recommended accumulate rat...

[5 rows x 4 columns]


In [0]:
# df.set_index("Timestamp" ,inplace = True)
# print(df.head())

                                                       ImgSrc  ...                                               Text
Timestamp                                                      ...                                                   
2019-06-15  https://static-news.moneycontrol.com/static-mc...  ...  Investors may tender their shares to enjoy ret...
2019-06-14  https://static-news.moneycontrol.com/static-mc...  ...  All eyes remain on the crude movement as well ...
2019-06-14  https://static-news.moneycontrol.com/static-mc...  ...  The European agency inspected Bengaluru unit i...
2019-06-14  https://static-news.moneycontrol.com/static-mc...  ...  Prabhudas Lilladher recommended accumulate rat...
2019-06-14  https://static-news.moneycontrol.com/static-mc...  ...  Prabhudas Lilladher recommended accumulate rat...

[5 rows x 3 columns]


In [0]:
df.Title[0]

'Wipro buyback likely to increase EPS, investors may tender their shares'

In [0]:
df[df['Title'].str.contains(" ")]

In [0]:
from pandas_datareader import data 

In [0]:
dataset = data.DataReader('infy' , 'yahoo' ,datetime.datetime(2018,11,1),datetime.datetime.now())

In [0]:
# Create a list of the opening prices and their corresponding daily headlines from the news
price = []
headlines = []

# for i in range(len(dataset)):
#   day = []
#   date = dataset.index[i].strftime(myFormat)
#   for j in range(len(df)):
#     if df.index[j] == date :
#       day.append(df.Title[j])
  
#   print(day[0])      
#   headlines.append(day)
#   price.append(dataset.Close[i])
#   #print(headlines)
  
for i in range (0,len(dataset)):
  day = []
  temp = []
  date = dataset.index[i].strftime(myFormat)
  for j in range(0 , len(df)):
    if df.index[j] == date:
      day.append(df.Title[j])
  print(day)
  headlines.append(day)  
  price.append(dataset.Close[i])

In [0]:
print(len(headlines))
print(len(price))

154
154


In [0]:
print(headlines)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['Post Q3, which multi-category consumer durables stock should you choose?', 'NELCO gains 1% on in-flight & maritime connectivity licence from DoT', 'D-Street Buzz: PSU banks extend gains led by PNB; Axis Bank hits new 52-week high, Zee Entertainment drags', 'Hold Supreme Industries; target of Rs 1018: Geojit', 'Hold Escorts; target of Rs 680: Geojit', 'Hold V-GUARD Industries; target of Rs 207: Geojit', 'Hold Aarti Industries; target of Rs 1787: Geojit', 'Hold Gabriel India; target of Rs 155: Geojit', 'Sugar stocks trade mixed after Cabinet nod for additional funds to sugar mills', 'Ideas for Profit | Bodal Chemicals: Vertical integration, better product

In [0]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}


In [0]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'0,0', '00', text) 
    text = re.sub(r'[_"\-;%()|.,+&=*%.,!?:#@\[\]]', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'\$', ' $ ', text)
    text = re.sub(r'u s ', ' united states ', text)
    text = re.sub(r'u n ', ' united nations ', text)
    text = re.sub(r'u k ', ' united kingdom ', text)
    text = re.sub(r'j k ', ' jk ', text)
    text = re.sub(r' s ', ' ', text)
    text = re.sub(r' yr ', ' year ', text)
    text = re.sub(r' l g b t ', ' lgbt ', text)
    text = re.sub(r'0km ', '0 km ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [0]:
import re
from nltk.corpus import stopwords
import nltk
# nltk.download('stopwords')
# Clean the headlines
clean_headlines = []

for daily_headlines in headlines[0]:
  clean_daily_headlines = []
  print(daily_headlines)
  
  
#   for headline in daily_headlines:
#     #txt = clean_text(headline)
#     #print(txt)
#     clean_daily_headlines.append(txt)
# #   print(clean_daily_headlines)
#   clean_headlines.append(clean_daily_headlines)

In [0]:
clean_headlines[0]

[]

News Collection from reuters

In [0]:
from bs4 import BeautifulSoup
import datetime
import pickle
import requests

In [0]:
stocks = ['AAPL.OQ' , 'AMZN.OQ' , 'KO.N' , 'TSLA.O' ,'INFY.NS' ]
print(stocks[-1])

INFY.NS


In [0]:
from pandas_datareader import data 
dataset = data.DataReader('infy' , 'yahoo' ,datetime.datetime(2019,6,1),datetime.datetime.now())

In [0]:
print(dataset.index[10])

2019-06-17 00:00:00


In [0]:
start_date = dataset.index[6]
end_date = dataset.index[-1]
print(start_date)
print(end_date)

2019-06-11 00:00:00
2019-06-17 00:00:00


In [0]:
# def getNewsFor(date):
#   file = open('news/' + date.strftime('%Y-%m-%d') + '.csv', 'w')
#   print('Getting news for ' + date.strftime('%Y-%m-%d'))
#   for i in range(len(stocks)):
#     query = 'http://www.reuters.com/finance/stocks/companyNews?symbol=' + stocks[i] + '&date=' + format(date.month, '02d') + format(date.day, '02d') + str(date.year)
#     print('Getting news for ' + stocks[i])

#     response = requests.get(query)
#     soup = BeautifulSoup(response.text, "html.parser")
#     divs = soup.findAll('div', {'class': 'feature'})
#     print('Found ' + str(len(divs)) + ' articles.')

#     if(len(divs) == 0):
#       continue

#     data = u''
#     for div in divs:
#       data = data.join(div.findAll(text=True))
#       file.write(stocks[i])
#       file.write(',')
#       file.write(data.replace('\n', ' '))
#       file.write('\n')
#   file.close()

In [0]:
# #data = pd.DataFrame(columns = ['Stock' , 'News'])
# while(start_date <= end_date):
#   print(start_date)
#   getNewsFor(start_date)
#   start_date += datetime.timedelta(days = 1)
  

In [0]:
mkdir news

In [0]:
cd news

/content/news


In [0]:
ls

sample_data/


In [0]:
cd ..

/content


In [0]:
rm -r news

In [0]:
while(start_date <= end_date):
  date = start_date
  file = open('/contents'  +  date.strftime('%Y-%m-%d') + '.csv', 'w')
  for i in range(len(stocks)):  
    query = 'https://www.reuters.com/finance/stocks/company-news/' + stocks[i] + '?date=' + format(date.month, '02d') + format(date.day, '02d') + str(date.year);
    print(query)
    response = requests.get(query)
    soup = BeautifulSoup(response.text, "html.parser")
    divs = soup.findAll('div', {'class': 'feature'})
    print('Found ' + str(len(divs)) + ' articles. For ' + stocks[i])

    for item in divs:
      file.write(stocks[i])
      file.write(',')
      file.write(item.find("h2").find("a").string)
      file.write('\n')
  file.close()
  start_date += datetime.timedelta(days = 1) 

https://www.reuters.com/finance/stocks/company-news/AAPL.OQ?date=06112019
Found 3 articles. For AAPL.OQ
https://www.reuters.com/finance/stocks/company-news/AMZN.OQ?date=06112019
Found 3 articles. For AMZN.OQ
https://www.reuters.com/finance/stocks/company-news/KO.N?date=06112019
Found 0 articles. For KO.N
https://www.reuters.com/finance/stocks/company-news/TSLA.O?date=06112019
Found 0 articles. For TSLA.O
https://www.reuters.com/finance/stocks/company-news/INFY.NS?date=06112019
Found 0 articles. For INFY.NS
https://www.reuters.com/finance/stocks/company-news/AAPL.OQ?date=06122019
Found 2 articles. For AAPL.OQ
https://www.reuters.com/finance/stocks/company-news/AMZN.OQ?date=06122019
Found 1 articles. For AMZN.OQ
https://www.reuters.com/finance/stocks/company-news/KO.N?date=06122019
Found 0 articles. For KO.N
https://www.reuters.com/finance/stocks/company-news/TSLA.O?date=06122019
Found 4 articles. For TSLA.O
https://www.reuters.com/finance/stocks/company-news/INFY.NS?date=06122019
Found 

In [0]:
date = dataset.index[5]
query = 'https://www.reuters.com/finance/stocks/company-news/' + stocks[-1] + '?date=' + format(date.month, '02d') + format(date.day, '02d') + str(date.year);
print(query)
response = requests.get(query)
soup = BeautifulSoup(response.text, "html.parser")
divs = soup.findAll('div', {'class': 'feature'})
print('Found ' + str(len(divs)) + ' articles. For ' + stocks[-1])

for item in divs:
  print(item.find("h2").find("a").string)


https://www.reuters.com/finance/stocks/company-news/INFY.NS?date=06102019
Found 0 articles. For INFY.NS


In [0]:
cd content/

/content


In [0]:
ls

sample_data/


In [0]:
rm AAPL_2019-06-14.csv

In [0]:
stocks = ['AAPL.OQ' , 'AMZN.OQ' , 'KO.N' , 'TSLA.O' ,'INFY.NS' ]
print(stocks[-1])

bin/      dev/   lib32/  opt/   sbin/   tensorflow-2.0.0a0/  var/
boot/     etc/   lib64/  proc/  srv/    tmp/
content/  home/  media/  root/  swift/  tools/
datalab/  lib/   mnt/    run/   sys/    usr/


In [0]:
df = pd.DataFrame(columns = ['Date' , 'Ticker' , 'News'])
c = 0
while(start_date <= end_date):
  date = start_date
  #file = open('/contents'  +  date.strftime('%Y-%m-%d') + '.csv', 'w')
  for i in range(len(stocks)):  
    query = 'https://www.reuters.com/finance/stocks/company-news/' + stocks[i] + '?date=' + format(date.month, '02d') + format(date.day, '02d') + str(date.year);
    print(query)
    response = requests.get(query)
    soup = BeautifulSoup(response.text, "html.parser")
    divs = soup.findAll('div', {'class': 'feature'})
    print('Found ' + str(len(divs)) + ' articles. For ' + stocks[i])

    for item in divs:
      df.loc[c] = [
                    str(date) ,
                    str(stocks[i]),
                    str(item.find("h2").find("a").string)
                  ]
      c += 1
  start_date += datetime.timedelta(days = 1) 

https://www.reuters.com/finance/stocks/company-news/AAPL.OQ?date=06112019
Found 3 articles. For AAPL.OQ
https://www.reuters.com/finance/stocks/company-news/AMZN.OQ?date=06112019
Found 3 articles. For AMZN.OQ
https://www.reuters.com/finance/stocks/company-news/KO.N?date=06112019
Found 0 articles. For KO.N
https://www.reuters.com/finance/stocks/company-news/TSLA.O?date=06112019
Found 0 articles. For TSLA.O
https://www.reuters.com/finance/stocks/company-news/INFY.NS?date=06112019
Found 0 articles. For INFY.NS
https://www.reuters.com/finance/stocks/company-news/AAPL.OQ?date=06122019
Found 2 articles. For AAPL.OQ
https://www.reuters.com/finance/stocks/company-news/AMZN.OQ?date=06122019
Found 1 articles. For AMZN.OQ
https://www.reuters.com/finance/stocks/company-news/KO.N?date=06122019
Found 0 articles. For KO.N
https://www.reuters.com/finance/stocks/company-news/TSLA.O?date=06122019
Found 4 articles. For TSLA.O
https://www.reuters.com/finance/stocks/company-news/INFY.NS?date=06122019
Found 

In [0]:
list_data = list_data[1:]

In [0]:
print(list_data)

['TSLA.O', 'TSLA.OQ', 'TSLA.BE', 'TSLA.D', 'TSLA.DE', 'TSLA.DF', 'TSLA.F', 'TSLA.H', 'TSLA.MU', 'TSLA.MW', 'TSLA.P', 'TSLA.PH', 'TXLZF.PK']


In [0]:
print(list_data)

['TSLA.O', 'TSLA.OQ', 'TSLA.A', 'TSLA.BA', 'TSLA.BE', 'TSLA.BN', 'TSLA.C', 'TSLA.D', 'TSLA.DE', 'TSLA.DF', 'TSLA.F', 'TSLA.H', 'TSLA.HA', 'TSLA.LM', 'TSLA.MU', 'TSLA.MW', 'TSLA.MX', 'TSLA.P', 'TSLA.PH', 'TSLA.S', 'TSLA.VI', 'TSLA.VIf', 'TSLA34.SA', 'TSLA34F.SA', 'TSLAc.BA', 'TSLAd.BA', 'TSLAh.BA', 'TSLAm.BA', 'TSLAn.BA', 'TSLAUSD.S', 'TSLAv.BA']


Google news data and analysis

In [1]:
ls

sample_data/
